49 文中のすべての名詞句のペアを結ぶ最短係り受けパスを抽出せよ．ただし，名詞句ペアの文節番号がi とj （i<j ）のとき，係り受けパスは以下の仕様を満たすものとする．

問題48と同様に，パスは開始文節から終了文節に至るまでの各文節の表現（表層形の形態素列）を” -> “で連結して表現する 文節i とj に含まれる名詞句はそれぞれ，XとYに置換する また，係り受けパスの形状は，以下の2通りが考えられる．

文節i から構文木の根に至る経路上に文節j が存在する場合: 文節i から文節j のパスを表示 上記以外で，文節i と文節j から構文木の根に至る経路上で共通の文節k で交わる場合: 文節i から文節k に至る直前のパスと文節j から文節k に至る直前までのパス，文節k の内容を” | “で連結して表示 「ジョン・マッカーシーはAIに関する最初の会議で人工知能という用語を作り出した。」という例文を考える． CaboChaを係り受け解析に用いた場合，次のような出力が得られると思われる．

Xは | Yに関する -> 最初の -> 会議で | 作り出した Xは | Yの -> 会議で | 作り出した Xは | Yで | 作り出した Xは | Yという -> 用語を | 作り出した Xは | Yを | 作り出した Xに関する -> Yの Xに関する -> 最初の -> Yで Xに関する -> 最初の -> 会議で | Yという -> 用語を | 作り出した Xに関する -> 最初の -> 会議で | Yを | 作り出した Xの -> Yで Xの -> 会議で | Yという -> 用語を | 作り出した Xの -> 会議で | Yを | 作り出した Xで | Yという -> 用語を | 作り出した Xで | Yを | 作り出した Xという -> Yを

In [ ]:
from itertools import combinations
import re

sentence = sentences[2]
nouns = []
for i, chunk in enumerate(sentence.chunks):
  if [morph for morph in chunk.morphs if morph.pos == "名詞"]:
    nouns.append(i)
for i, j in combinations(nouns, 2):
  path_I = []
  path_J = []
  while i != j:
    if i < j: #文節iの構文木経路上に文節jが存在する場合
      path_I.append(i)
      i = sentence.chunks[i].dst
    else: #文節iの構文木経路上に文節jがない場合
      path_J.append(j)
      j = sentence.chunks[j].dst

  if len(path_J) == 0: # 文節Iの構文木上に文節Jが存在する場合
    X = "X" + "".join([morph.surface for morph in sentence.chunks[path_I[0]].morphs if morph.pos != "名詞" and morph.pos != "記号"])
    Y = "Y" +  "".join([morph.surface for morph in sentence.chunks[i].morphs if morph.pos != "名詞" and morph.pos != "記号"])
    chunk_X = re.sub("X+", "X", X)
    chunk_Y = re.sub("Y+", "Y", Y)
    path_ItoJ = [chunk_X] + ["".join(morph.surface for n in path_I[1:] for morph in sentence.chunks[n].morphs)] + [chunk_Y]
    print(" -> ".join(path_ItoJ))
  else: # 文節Iの構文木上に文節Jが存在しない場合
    X = "X" + "".join([morph.surface for morph in sentence.chunks[path_I[0]].morphs if morph.pos != "名詞" and morph.pos != "記号"])
    Y = "Y" + "".join([morph.surface for morph in sentence.chunks[path_J[0]].morphs if morph.pos != "名詞" and morph.pos != "記号"])
    chunk_X = re.sub("X+", "X", X)
    chunk_Y = re.sub("Y+", "Y", Y)
    chunk_k = "".join([morph.surface for morph in sentence.chunks[i].morphs if morph.pos != "記号"])
    path_X = [chunk_X] + ["".join(morph.surface for n in path_I[1:] for morph in sentence.chunks[n].morphs if morph.pos != "記号")]
    path_Y = [chunk_Y] + ["".join(morph.surface for n in path_J[1: ]for morph in sentence.chunks[n].morphs if morph.pos != "記号")]
    print(" | ".join(["->".join(path_X), "->".join(path_Y), chunk_k]))